In [1]:
# Cell 1: 环境变量必须在 import 之前设置
import os
os.environ['ADMIN_LOG_LEVEL'] = 'DEBUG'
os.environ['CURVE_CACHE_DIR'] = r'C:\temp\curve_cache'  # 或 '/tmp/curve_cache'，务必绝对路径

# 可选：本地调试时临时放行某些接口，避免 device_gate 拦截（修改过 adminserver 后需重启 Kernel）
# 你已经加了 /admin/api/log-test 放行，如果还要打 bind-model/inspect，也可以在代码里加入放行。

In [3]:
# Cell 2: 启动服务（同进程、可捕获异常、可优雅停止）
import sys, threading, time, traceback, requests
from werkzeug.serving import make_server

from adminserver import app  # 此时会执行 _setup_logging，并把 flask.app 日志绑到 stdout

class FlaskServerThread(threading.Thread):
    def __init__(self, app, host='127.0.0.1', port=6001):
        super().__init__(daemon=False)            # 非 daemon，异常不会悄悄吞掉
        self.app = app
        self.host = host
        self.port = port
        self.srv = None
        self.exc = None

    def run(self):
        try:
            self.srv = make_server(self.host, self.port, self.app)
            self.app.logger.info("Werkzeug WSGI server starting on http://%s:%s", self.host, self.port)
            self.srv.serve_forever()
        except Exception as e:
            self.exc = e
            traceback.print_exc()  # 把异常打回 Notebook 输出，便于诊断

    def stop(self):
        if self.srv:
            self.app.logger.info("Shutting down server ...")
            self.srv.shutdown()

server = FlaskServerThread(app, host='127.0.0.1', port=6001)
server.start()

# 确认服务可用（不使用 .json()，先看实际内容，避免被 403/302 HTML 误导）
time.sleep(0.5)
r = requests.get("http://127.0.0.1:6001/admin/api/log-test", allow_redirects=False)
print("status:", r.status_code, "content-type:", r.headers.get("Content-Type"))
print("body[:200]:", r.text[:200])

[2025-10-30 09:51:43,601] INFO adminserver: Logging configured with StreamHandler (level=DEBUG)
[2025-10-30 09:51:43,603] INFO adminserver: Registered routes:
POST       /admin/api/calib/bind-model  -> data_mgmt.api_bind_model_to_perf
POST       /admin/api/calib/bind-model  -> calib_admin.api_bind_model_to_perf
GET        /admin/api/calib/bindings  -> calib_admin.api_admin_calib_bindings_by_mid_cid
GET        /admin/api/calib/cache/inspect  -> calib_admin.api_calib_cache_inspect
GET        /admin/api/calib/preview  -> calib_admin.api_calib_preview
GET        /admin/api/calib/preview/debug  -> calib_admin.api_calib_preview_debug
GET        /admin/api/calib/rpm-noise  -> calib_admin.api_admin_calib_rpm_noise
POST       /admin/api/calib/upload_zip  -> calib_admin.api_calib_upload_zip
GET        /admin/api/data/batch/models  -> data_mgmt.api_batch_models
POST       /admin/api/data/batch/search  -> data_mgmt.api_batch_search
POST       /admin/api/data/batch/status  -> data_mgmt.api_batch_st